# Linear solvers and preconditioners

## Initialization

In [ ]:
.I /root/install/do-conf-tp-serial/include/

In [ ]:
.L libteuchosparameterlist

In [ ]:
.L libtpetra

In [ ]:
.L libifpack2

In [ ]:
.L libbelos

## Header inclusions and typedefs

In [ ]:
#include <Tpetra_CrsMatrix.hpp>
#include <Tpetra_Vector.hpp>
#include <Tpetra_Map_decl.hpp>

In [ ]:
#include <BelosConfigDefs.hpp>
#include <BelosLinearProblem.hpp>
#include <BelosBlockCGSolMgr.hpp>
#include <BelosPseudoBlockCGSolMgr.hpp>
#include <BelosBlockGmresSolMgr.hpp>
#include <BelosTpetraAdapter.hpp>

In [ ]:
typedef double                                      scalar_type;
typedef int                                         local_ordinal_type;
typedef int                                         global_ordinal_type;

// Convenient typedef's
typedef Tpetra::Operator<scalar_type,local_ordinal_type,global_ordinal_type>    operator_type;
typedef Tpetra::CrsMatrix<scalar_type,local_ordinal_type,global_ordinal_type>   crs_matrix_type;
typedef Tpetra::Vector<scalar_type,local_ordinal_type,global_ordinal_type>      vector_type;
typedef Tpetra::MultiVector<scalar_type,local_ordinal_type,global_ordinal_type> multivector_type;
typedef Tpetra::Map<local_ordinal_type,global_ordinal_type>                     driver_map_type;

typedef Belos::LinearProblem<scalar_type, multivector_type, operator_type> linear_problem_type;
typedef Belos::SolverManager<scalar_type, multivector_type, operator_type> belos_solver_manager_type;
typedef Belos::PseudoBlockCGSolMgr<scalar_type, multivector_type, operator_type> belos_pseudocg_manager_type;
typedef Belos::BlockGmresSolMgr<scalar_type, multivector_type, operator_type> belos_gmres_manager_type;

## Create comm object

MPI is deactivated for the CLING tutorials. Therefore, the default communicator is a Serial communicator.

In [ ]:
#include <Teuchos_DefaultComm.hpp>

In [ ]:
auto comm = Teuchos::DefaultComm<int>::getComm();

## Define linear problem

### Helper method to create matrix $A$

In [ ]:
const Tpetra::global_size_t numGblIndices = 50;

In [ ]:
Teuchos::RCP<const crs_matrix_type> createMatrix(const Teuchos::RCP<driver_map_type>& map)
{
    const size_t numMyElements = map->getNodeNumElements ();
    Teuchos::RCP<crs_matrix_type> A (new crs_matrix_type (map, 3));
    
    const scalar_type two = static_cast<scalar_type> (2.0);
    const scalar_type negOne = static_cast<scalar_type> (-1.0);
    for (local_ordinal_type lclRow = 0; lclRow < static_cast<local_ordinal_type> (numMyElements); ++lclRow) 
    {
        const global_ordinal_type gblRow = map->getGlobalElement (lclRow);
        // A(0, 0:1) = [2, -1]
        if (gblRow == 0) 
        {
            A->insertGlobalValues (gblRow,
             Teuchos::tuple<global_ordinal_type> (gblRow, gblRow + 1),
             Teuchos::tuple<scalar_type> (two, negOne));
        }
        // A(N-1, N-2:N-1) = [-1, 2]
        else if (static_cast<Tpetra::global_size_t> (gblRow) == numGblIndices - 1) 
        {
            A->insertGlobalValues (gblRow,
             Teuchos::tuple<global_ordinal_type> (gblRow - 1, gblRow),
             Teuchos::tuple<scalar_type> (negOne, two));
        }
        // A(i, i-1:i+1) = [-1, 2, -1]
        else {
            A->insertGlobalValues (gblRow,
             Teuchos::tuple<global_ordinal_type> (gblRow - 1, gblRow, gblRow + 1),
             Teuchos::tuple<scalar_type> (negOne, two, negOne));
        }
    }
    // Tell the sparse matrix that we are done adding entries to it.
    A->fillComplete ();
    return A;
}

### Create `Tpetra::Map` object used to generate matrix $A$

In [ ]:
Teuchos::RCP<driver_map_type> map = Teuchos::rcp(new driver_map_type(numGblIndices,0,comm));

In [ ]:
std::cout << map->description() << std::endl;

Call helper function creating the tridiagonal problem matrix $A$

In [ ]:
Teuchos::RCP<const crs_matrix_type> A = createMatrix(map);

Define right hand-side vector $B$ and solution vector $X$. For this simple (serial) example we do not need to distinguish between row/range and column/domain maps.

In [ ]:
Teuchos::RCP<multivector_type> X = Teuchos::rcp(new multivector_type(map,1));
Teuchos::RCP<multivector_type> B = Teuchos::rcp(new multivector_type(map,1));

Initialize solution vector $X=0$ and fill right hand-side vector $B$ with some random values.

In [ ]:
X->putScalar((scalar_type) 0.0);
B->randomize();

In [ ]:
Teuchos::RCP<linear_problem_type> Problem = Teuchos::rcp(new linear_problem_type(A, X, B));

The following command initializes internals of the `Belos::LinearProblem` class. See [the documentation](https://trilinos.org/docs/dev/packages/belos/doc/html/classBelos_1_1LinearProblem.html#afc39e9701a7207af47031859d15b3e8a) for details.

In [ ]:
Problem->setProblem();

In [ ]:
#include <Teuchos_ParameterList.hpp>

### Create a Belos solver

In [ ]:
Teuchos::RCP<Teuchos::ParameterList> belosList = Teuchos::rcp(new Teuchos::ParameterList());

In [ ]:
belosList->set("Maximum Iterations",    1000); // Maximum number of iterations allowed
belosList->set("Convergence Tolerance", 1e-8);    // Relative convergence tolerance requested
belosList->set("Verbosity",             Belos::Errors + Belos::Warnings + Belos::StatusTestDetails);
belosList->set("Output Frequency",      10);
belosList->set("Output Style",          Belos::Brief);
belosList->set("Implicit Residual Scaling", "None");

In [ ]:
Belos::SolverFactory<scalar_type, multivector_type, operator_type> factory;

In [ ]:
Teuchos::RCP<Belos::SolverManager<scalar_type, multivector_type, operator_type> > solver = 
factory.create ("GMRES", belosList);

Make the Belos solver aware of the `Belos::LinearProblem` instance that is to be solved

In [ ]:
solver->setProblem (Problem);

Solve the linear problem. The result is stored in the solution vector $X$.

In [ ]:
solver->solve();

We have not set a preconditioner in the `Belos::LinearProblem` class. Therefore, the convergence is very slow. After $50$ iterations, the Krylov method of course has found the exact solution.

### Create a ILUT preconditioner

In [ ]:
#include <Ifpack2_Factory.hpp>

In [ ]:
// The name of the type of preconditioner to use.
std::string precondType = "ILUT";

// Ifpack2 expects arguments of type 'double' here, regardless of
// the scalar or magnitude types of the entries of the sparse
// matrix.
const double fillLevel = 2.0;
const double dropTol = 0.0;
const double absThreshold = 0.1;

Teuchos::ParameterList ifpack2Params;
ifpack2Params.set ("fact: ilut level-of-fill", fillLevel);
ifpack2Params.set ("fact: drop tolerance", dropTol);
ifpack2Params.set ("fact: absolute threshold", absThreshold);

In [ ]:
typedef Ifpack2::Preconditioner<scalar_type, local_ordinal_type, 
                  global_ordinal_type, Tpetra::Map<local_ordinal_type,global_ordinal_type>::node_type> 
                  prec_type;

Use the `Ifpack2::Factory` factory class to create the preconditioner operator

In [ ]:
Teuchos::RCP<prec_type> prec;

In [ ]:
Ifpack2::Factory prec_factory;
prec = prec_factory.create (precondType, A);
prec->setParameters (ifpack2Params);

In [ ]:
prec->initialize();

In [ ]:
prec->compute();

### Solve the linear system with the ILUT preconditioner

Reset the solution vector $X=0$ for the iterative solver.

In [ ]:
X->putScalar((scalar_type) 0.0);

Tell the `Belos::LinearProblem` instance to use the ILUT preconditioning operator as preconditioner (right-preconditioner). Call `Belos::LinerProblem::setProblem()` to reset all internals of the linear problem before solving it again.

In [ ]:
Problem->setRightPrec(prec);
Problem->setProblem();

Solve the linear problem again with the linear solver object.

In [ ]:
solver->solve();

We solved the same problem as before (note: we have not changed the RHS vector $B$ and reset the initial solution vector $X$ to be zero in both cases). The only difference is the preconditioning: without preconditioner, the solver does not find a solution with a residual tolerance of 1e-8 within $50$ iterations. With the preconditioner, the solver converges within $18$ iterations